In [2]:
from openbb_terminal.sdk import openbb
import plotly.graph_objects as go
import yfinance as yf
import pandas as pd
import plotly.express as px
import numpy as np


In [4]:
# want to find aggregate closes for 5m, 30m, and 1hr during trading hrs 8-10
# what % of time is the rsi below/above the 50 
import datetime
current_timestamp = datetime.datetime.now()


formatted_date = current_timestamp.strftime("%Y-%m-%d")
print("Current timestamp:", formatted_date)

Current timestamp: 2024-05-05


In [5]:

df = openbb.futures.historical(symbols='GC', start_date='2000-08-30', end_date= formatted_date)

df = df.reset_index().rename(columns={'index': 'Date'})
# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
5936,2024-04-29,2334.800049,2345.500000,2327.899902,2345.399902,2345.399902,405
5937,2024-04-30,2322.500000,2322.500000,2291.399902,2291.399902,2291.399902,113
5938,2024-05-01,2287.699951,2327.300049,2283.800049,2299.899902,2299.899902,42
5939,2024-05-02,2321.699951,2324.699951,2292.300049,2299.199951,2299.199951,56
5940,2024-05-03,2300.000000,2301.000000,2277.600098,2299.000000,2299.000000,56


In [6]:
# df['rsi'] = openbb.ta.rsi(df['Adj Close'], 14).reset_index(drop=True)
df['rsi'] = openbb.ta.rsi(df['Adj Close'], 14)
# Assuming df is your DataFrame
# df = df.drop(columns=df.columns[0])

df.head(28)


,Date,Open,High,Low,Close,Adj Close,Volume,rsi
0,2000-08-30,273.899994,273.899994,273.899994,273.899994,273.899994,0,NaN
1,2000-08-31,274.799988,278.299988,274.799988,278.299988,278.299988,0,NaN
2,2000-09-01,277.000000,277.000000,277.000000,277.000000,277.000000,0,NaN
3,2000-09-05,275.799988,275.799988,275.799988,275.799988,275.799988,2,NaN
4,2000-09-06,274.200012,274.200012,274.200012,274.200012,274.200012,0,NaN
5,2000-09-07,274.000000,274.000000,274.000000,274.000000,274.000000,125,NaN
6,2000-09-08,273.299988,273.299988,273.299988,273.299988,273.299988,0,NaN
7,2000-09-11,273.100006,273.100006,273.100006,273.100006,273.100006,0,NaN
8,2000-09-12,272.899994,272.899994,272.899994,272.899994,272.899994,0,NaN
9,2000-09-13,272.799988,272.799988,272.799988,272.799988,272.799988,0,NaN


In [7]:
df.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
rsi                 float64
dtype: object

In [8]:
df.loc[(df['rsi'] > 60), 'above'] = True
df['above'].fillna(False, inplace=True)

In [9]:
df.loc[(df['rsi'] < 40), 'below'] = True
df['below'].fillna(False, inplace=True)

In [10]:
df['group'] = (df['above'] != df['above'].shift()).cumsum()

# Filter out the groups where 'value' is not above the threshold
above_groups = df[df['above']].groupby('group')
above_groups_df = above_groups.apply(lambda x: x.reset_index(drop=True))


In [11]:
above_groups_df.head()

Date        Open        High         Low       Close  \
group                                                                
2     0 2000-09-27  278.399994  278.399994  278.399994  278.399994   
4     0 2000-11-27  270.100006  270.100006  270.100006  270.100006   
      1 2000-11-28  269.799988  269.799988  269.799988  269.799988   
6     0 2000-12-06  271.000000  274.299988  270.600006  274.299988   
      1 2000-12-07  274.299988  276.000000  272.600006  273.200012   

          Adj Close  Volume        rsi  above  below  group  
group                                                        
2     0  278.399994    3511  67.840153   True  False      2  
4     0  270.100006       0  62.823342   True  False      4  
      1  269.799988   28696  61.361317   True  False      4  
6     0  274.299988      81  65.957883   True  False      6  
      1  273.200012      36  62.335740   True  False      6

In [12]:
df['group_b'] = (df['below'] != df['below'].shift()).cumsum()

# Filter out the groups where 'value' is not above the threshold
below_groups = df[df['below']].groupby('group_b')
below_groups_df = below_groups.apply(lambda x: x.reset_index(drop=True))

below_groups_df.head()




Date        Open        High         Low       Close  \
group_b                                                                
2       0 2000-09-20  269.000000  269.000000  269.000000  269.000000   
        1 2000-09-21  270.299988  270.299988  270.299988  270.299988   
4       0 2000-10-06  269.799988  270.200012  269.299988  269.299988   
6       0 2000-10-25  266.200012  267.100006  266.000000  266.200012   
        1 2000-10-26  266.000000  267.000000  264.899994  264.899994   

            Adj Close  Volume        rsi  above  below  group  group_b  
group_b                                                                 
2       0  269.000000       0  24.271483  False   True      1        2  
        1  270.299988       0  34.634546  False   True      1        2  
4       0  269.299988       2  38.617084  False   True      3        4  
6       0  266.200012      20  36.581517  False   True      3        6  
        1  264.899994       0  34.289787  False   True      3        6

In [13]:
# Calculate the duration of each stretch
durations = above_groups['Date'].agg(['min', 'max'])
durations['duration'] = durations['max'] - durations['min']
durations.head()

,min,max,duration
group,,,
2,2000-09-27,2000-09-27,0 days
4,2000-11-27,2000-11-28,1 days
6,2000-12-06,2000-12-07,1 days
8,2000-12-21,2000-12-27,6 days
10,2001-02-27,2001-02-27,0 days


In [14]:
above = above_groups['Date'].agg(['min', 'max'])
above['duration'] = above['max'] - above['min']

In [15]:
below = below_groups['Date'].agg(['min', 'max'])
below['duration'] = below['max'] - below['min']

In [16]:
# Convert duration to a suitable format (e.g., days)
durations['duration_days'] = durations['duration'].dt.days + 1  # +1 to include the start day
below['duration_days'] = below['duration'].dt.days + 1
above['duration_days'] = above['duration'].dt.days + 1

In [17]:
rows = above.loc[above['duration_days'] > 30]
rows

,min,max,duration,duration_days
group,,,,
64,2002-12-12,2003-02-07,57 days,58
66,2003-05-01,2003-06-03,33 days,34
94,2003-12-01,2004-01-14,44 days,45
114,2004-11-04,2004-12-07,33 days,34
138,2005-12-28,2006-02-06,40 days,41
140,2006-03-27,2006-05-17,51 days,52
166,2007-09-04,2007-11-12,69 days,70
172,2007-12-26,2008-02-04,40 days,41
224,2009-11-03,2009-12-03,30 days,31


In [33]:

# Create a new column for x-axis labels showing min and max date
durations['label'] = durations.apply(lambda x: f"{x['min'].strftime('%Y-%m-%d')} to {x['max'].strftime('%Y-%m-%d')}", axis=1)


# Plot with Plotly
fig = px.bar(
    durations,
    x='label',
    y='duration_days',
    labels={'label': 'Date Range', 'duration_days': 'Duration (Days)'},
    color='duration_days'
)

# Customize layout
fig.update_layout(
    title='Duration of Value Above Threshold by Date Range',
    xaxis_title='',  # No x-axis title (to place it below the chart)
    yaxis_title='',  # Remove y-axis label
    xaxis=dict(
        tickmode='array',
        tickvals=[],
        showticklabels=False  # Hide x-axis tick labels
    ),
    yaxis=dict(
        title=''  # Remove y-axis label
    ),
    # Place x-axis label below the chart
    margin=dict(t=80),  # Increase top margin to make space for x-axis label
    annotations=[
        dict(
            xref='paper',
            yref='paper',
            x=0.5,
            y=-0.1,  # Adjust this value to position the x-axis label below the chart
            text='Date Range',  # X-axis label
            showarrow=False,
            font=dict(size=12)  # Adjust font size if needed
        )
    ]
)

# Show the plot
fig.show()



In [34]:

# Create a new column for x-axis labels showing min and max date
above['label'] = above.apply(lambda x: f"{x['min'].strftime('%Y-%m-%d')} to {x['max'].strftime('%Y-%m-%d')}", axis=1)


# Plot with Plotly
fig = px.bar(
    above,
    x='label',
    y='duration_days',
    labels={'label': 'Date Range', 'duration_days': 'Duration (Days)'},
    color='duration_days'
)

# Customize layout
fig.update_layout(
    title='Duration of Value Above Threshold by Date Range',
    xaxis_title='',  # No x-axis title (to place it below the chart)
    yaxis_title='',  # Remove y-axis label
    xaxis=dict(
        tickmode='array',
        tickvals=[],
        showticklabels=False  # Hide x-axis tick labels
    ),
    yaxis=dict(
        title=''  # Remove y-axis label
    ),
    # Place x-axis label below the chart
    margin=dict(t=80),  # Increase top margin to make space for x-axis label
    annotations=[
        dict(
            xref='paper',
            yref='paper',
            x=0.5,
            y=-0.1,  # Adjust this value to position the x-axis label below the chart
            text='Date Range',  # X-axis label
            showarrow=False,
            font=dict(size=12)  # Adjust font size if needed
        )
    ]
)

# Show the plot
fig.show()



In [35]:

# Create a new column for x-axis labels showing min and max date
below['label'] = below.apply(lambda x: f"{x['min'].strftime('%Y-%m-%d')} to {x['max'].strftime('%Y-%m-%d')}", axis=1)


# Plot with Plotly
fig = px.bar(
    below,
    x='label',
    y='duration_days',
    labels={'label': 'Date Range', 'duration_days': 'Duration (Days)'},
    color='duration_days'
)

# Customize layout
fig.update_layout(
    title='Duration of Value Below Threshold by Date Range',
    xaxis_title='',  # No x-axis title (to place it below the chart)
    yaxis_title='',  # Remove y-axis label
    xaxis=dict(
        tickmode='array',
        tickvals=[],
        showticklabels=False  # Hide x-axis tick labels
    ),
    yaxis=dict(
        title=''  # Remove y-axis label
    ),
    # Place x-axis label below the chart
    margin=dict(t=80),  # Increase top margin to make space for x-axis label
    annotations=[
        dict(
            xref='paper',
            yref='paper',
            x=0.5,
            y=-0.1,  # Adjust this value to position the x-axis label below the chart
            text='Date Range',  # X-axis label
            showarrow=False,
            font=dict(size=12)  # Adjust font size if needed
        )
    ]
)

# Show the plot
fig.show()

